In [4]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [5]:
from PIL import Image 
AmazingCredit = Image.open(r"./AmazingCredit.png")
GoodCredit = Image.open(r"./GoodCredit.png")
BadCredit = Image.open(r"./BadCredit.png")

import ipywidgets as widget

AmazingCredit =  widget.HTML('<div id="img" style="float:left">'
                    '<img style="vertical-align:middle" src="./AmazingCredit.png" alt="">'                       
                    '</div>')

GoodCredit =  widget.HTML('<div id="img" style="float:left">'
                    '<img style="vertical-align:middle" src="./GoodCredit.png" alt="">'                       
                    '</div>')

BadCredit =  widget.HTML('<div id="img" style="float:left">'
                    '<img style="vertical-align:middle" src="./BadCredit.png" alt="">'                       
                    '</div>')

In [6]:
data_loading_bay = {"avg_borrowRate" : [], "avg_amount_borrowed" : [], "num_borrows" : [], "num_prev_defaults" : [], "default_this_time" : []}

In [7]:
def eth_to_dict(input_address):
    graph_call = open("thing.sh", "w")
    address = input_address
    variable = '''curl 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2' --data-binary '{"query":"query MyQuery { users(where: {id: \\"''' + address + '''\\"}) { borrowHistory { borrowRate  borrowRateMode  amount  id  timestamp } liquidationCallHistory { collateralAmount  id  timestamp  } } } ","variables":{}}' > call_data.txt -s'''
    graph_call.write(r"{}".format(variable))
    graph_call.close()

    os.system("bash thing.sh")

    data_loading_bay = {"avg_borrowRate" : [], "avg_amount_borrowed" : [], "num_borrows" : [], "num_prev_defaults" : [], "default_this_time" : []}

    with open("call_data.txt") as json_file:
      all_data = json.load(json_file)
      data = all_data["data"]
      for user in data["users"]:
        if user["borrowHistory"] != []:
          num_borrows = 0
          total_amount = 0
          total_borrowRate = 0
          num_prev_defaults = 0
          for borrow in user["borrowHistory"]:
            num_borrows += 1
            total_amount += int(borrow["amount"])
            total_borrowRate += int(borrow["borrowRate"])
            last_borrow_id = borrow["id"]
          if user["liquidationCallHistory"] != []:
            for default in user["liquidationCallHistory"]:
              num_prev_defaults = 0
              for _ in user["liquidationCallHistory"]:
                num_prev_defaults += 1
              # if default["id"] == last_borrow_id:
                default_this_time = 1
                # num_prev_defaults -= 1
              # else:
              #   default_this_time = 0
          else:
            prev_defaults = 0
            default_this_time = 0

          data_loading_bay["avg_borrowRate"].append(total_borrowRate / num_borrows)
          data_loading_bay["avg_amount_borrowed"].append(total_amount / num_borrows)
          data_loading_bay["num_borrows"].append(num_borrows)
          data_loading_bay["num_prev_defaults"].append(num_prev_defaults)
          data_loading_bay["default_this_time"].append(default_this_time)
    return data_loading_bay

# CredAAVE

A tool allowing you to check the credit score of accounts for AAVE!

Built using GraphQL, AAVE, and PyTorch (Machine Learning)

[Video Demo](https://loom.com/)  
[GitHub](https://github.com/DicksonWu654/credaave)  

You can try out some examples here! I just randomly picked some:
- Great Score! 0x0000006daea1723962647b7e189d311d757fb793. They took out loans 24 times, never defaulted
- Good Score! 0x6375b32ac8c1ffd97b1eb105659872b2e308502a
- Bad Score! 0x0bf66bed1971055c5df68946d5ad094d0a9901b7. They took out >40 loans and defaulted on >30 of them!
- Bad Score! 0x57dc4a5d2786cb0fef99fa526b0ff5038e185460

You can sample your own by following the instructions here: https://gist.github.com/DicksonWu654/7a7510cbadccca3ebe8bb9597f4376eb

Improvements: Train on a much bigger dataset with many more features, label the data to be unsupervised learning, implement state of the art techniques, use explainable AI

-------

In [8]:
path = Path()
learn_inf = load_learner(path/'testing3.pkl', cpu=True)
txt_input = widgets.Text()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [82]:
def display_credit(cred):
    cred = 1 - cred
    cred -= 0.5
    cred *= 6
    cred = 1 / (1 + math.exp(-cred))
    if cred >= 0.9:
        print("Amazing credit score! This ETH Address only has a {:.2f}% chance of defaulting!".format((1 - cred) * 100))
        display(AmazingCredit)
    elif cred >=0.7:
        print("Good credit score! This ETH Address has a {:.2f}% chance of defaulting!".format((1 - cred) * 100))
        display(GoodCredit)
    else:
        print ("Bad credit score! This ETH Address has a {:.2f}% chance of defaulting!".format((1 - cred) * 100))
        display(BadCredit)

In [83]:
def on_text_change(change):
    lbl_pred.value = ''
    out_pl.clear_output()
    data_loading_bay = eth_to_dict(change)
    pred_df = pd.DataFrame(data_loading_bay)
    pred_df_true = pred_df.squeeze()
    cred = float(learn_inf.predict(pred_df_true)[2])
    return display_credit(cred)

In [87]:
display(VBox([widgets.Label('Input a valid Ethereum address:'), txt_input, out_pl, lbl_pred]))

In [88]:
button = widgets.Button(description='Run It!')

In [89]:
out = widgets.Output()
def on_button_clicked(_):
      with out:
        out.clear_output()
        on_text_change(str(txt_input)[12:-2])
button.on_click(on_button_clicked)
widgets.VBox([button, out])